In [ ]:
import sys
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import os

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

win_path = f'../features/4_winner/*.gz'
fname=''
# submit = pd.read_csv('../input/sample_submission.csv')
submit = []

HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

### OUTLIER SCORE

In [ ]:
import re
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

# outlierに対するスコアを出す
train = utils.read_df_pkl('../input/train0*')
out_ids = train.loc[train.target<-30, key].values
out_val = train.loc[train.target<-30, target].values

# outlier以外に対するスコアを出す
com_ids = train.loc[train.target>-30, key].values
com_val = train.loc[train.target>-30, target].values

path_list = glob.glob('../log_submit/01*csv')
stack_list = glob.glob('../stack/*gz')

out_dict = {}
for i, path in enumerate(tqdm(path_list)):
    tmp_path = path.replace(".", '-')
    cv = re.search(r'CV([^/.]*)_LB', tmp_path).group(1)[:-2].replace('.', '-')
    lb = re.search(r'LB([^/.]*).csv', tmp_path).group(1).replace('-', '.')
    stack_path = [s for s in stack_list if s.count(cv.replace('.', '-'))][0]
    
    df_pred = utils.read_pkl_gzip(stack_path)
    
    if len(df_pred.columns)>3:
        com_pred = df_pred[df_pred[key].isin(com_ids)]['pred_mean'].values
        out_pred = df_pred[df_pred[key].isin(out_ids)]['pred_mean'].values
    else:
        com_pred = df_pred[df_pred[key].isin(com_ids)]['prediction'].values
        out_pred = df_pred[df_pred[key].isin(out_ids)]['prediction'].values
        
    com_score = np.sqrt(mean_squared_error(com_val, com_pred))
    out_score = np.sqrt(mean_squared_error(out_val, out_pred))
    out_vals = {
        '1_CV' : cv.replace('-', '.'),
        '2_LB' : lb,
        '3_OUT': out_score,
        '4_COM': com_score,
               }
    out_dict[str(i)] = out_vals
    
result = pd.DataFrame(out_dict)
display(result.T)
sys.exit()
tmp = result.T['1_CV'][result.T['1_CV']=='3.65133231891838'].index
base = result[tmp]
comp = (result.astype('float').values - base.astype('float').values).T
df = pd.DataFrame(comp.T, columns=result.loc['1_CV', :], index=result.index).T
df.sort_values(by="3_OUT", inplace=True)
display(df)

In [100]:
import pandas as pd
import glob
import re
import sys

path_list = glob.glob('../log_submit/01*.csv')
path_list_2 = glob.glob('../check_submit/*.csv')
path_list += path_list_2
# path_list.append('../submit/0104_211_submit_lgb_rate0.01_101features_1seed_CV3.6502800495477397_LB.csv')
# stack_list = glob.glob('../stack/*gz')

tmp_list = []
for path in path_list:
    tmp = pd.read_csv(path)
    tmp_path = path.replace(".", '-')
    cv = re.search(r'CV([^/.]*)_LB', tmp_path).group(1)[:-2].replace('-', '.')
    lb = re.search(r'LB([^/.]*).csv', tmp_path).group(1).replace('-', '.')
    tmp.rename(columns={'target':f"CV{cv[:7]}_LB{lb}"}, inplace=True)
    tmp.set_index('card_id', inplace=True)
#     print([s for s in stack_list if s.count(cv.replace('.', '-'))][0])
    tmp_list.append(tmp.copy())
    
df = pd.concat(tmp_list, axis=1)
df.head()

,CV3.65355_LB3.694,CV3.64974_LB3.692,CV3.65034_LB3.692,CV3.64792_LB3.691,CV3.64778_LB3.691,CV3.65124_LB3.690,CV3.65022_LB3.690,CV3.65167_LB3.690,CV3.65133_LB3.689,CV3.64877_LB3.698,...,CV3.65263_LB3.694,CV3.65221_LB3.692,CV3.64973_LB3.692,CV3.64851_LB3.693,CV3.65275_LB,CV3.65133_LB,CV3.65116_LB,CV3.65085_LB,CV3.65101_LB,CV3.65263_LB
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_0ab67a22ab,-3.948606,-3.365106,-3.854378,-3.915047,-3.761760,-3.334345,-3.368916,-3.447426,-3.588557,-3.447698,...,-3.906811,-3.054859,-3.990014,-3.817082,-3.714998,-3.588557,-3.515814,-3.534043,-3.781980,-3.478843
C_ID_130fd0cbdd,-0.419410,-0.427927,-0.291161,-0.500042,-0.510945,-0.447128,-0.444178,-0.434026,-0.509478,-0.306966,...,-0.570409,-0.380678,-0.366399,-0.469466,-0.446702,-0.509478,-0.460639,-0.457304,-0.526015,-0.472827
C_ID_b709037bc5,-1.207178,-1.121759,-1.278443,-1.204693,-1.133613,-1.141604,-1.075473,-1.138398,-1.168318,-0.914125,...,-1.170335,-1.068673,-1.218352,-1.209258,-1.178001,-1.168318,-1.067751,-1.096126,-1.100735,-1.096073
C_ID_d27d835a9f,-0.108546,-0.160725,-0.110240,-0.154292,-0.130175,-0.155045,-0.149351,-0.154011,-0.156544,-0.048680,...,-0.111461,-0.150873,-0.190369,-0.082269,-0.139881,-0.156544,-0.138220,-0.154698,-0.143136,-0.144532
C_ID_2b5e3df5c2,-1.202471,-1.151264,-1.084603,-1.191651,-1.155295,-1.175872,-1.201367,-1.202798,-1.163820,-1.168852,...,-1.100262,-1.195107,-1.045499,-1.282403,-1.185871,-1.163820,-1.175591,-1.174802,-1.124315,-1.120419


In [79]:
df_corr = df.corr(method='pearson')
display(df_corr['CV3.65093_LB'].sort_values())

CV3.64851_LB3.693    0.979454
CV3.64923_LB3.691    0.985131
CV3.64877_LB3.698    0.987309
CV3.64792_LB3.691    0.991196
CV3.64778_LB3.691    0.991510
CV3.65221_LB3.692    0.991810
CV3.65263_LB3.694    0.992029
CV3.65034_LB3.692    0.994275
CV3.64973_LB3.692    0.994506
CV3.65355_LB3.694    0.994814
CV3.64974_LB3.692    0.995200
CV3.65124_LB3.690    0.996721
CV3.65001_LB3.689    0.996799
CV3.65167_LB3.690    0.996836
CV3.65133_LB         0.997445
CV3.65133_LB3.689    0.997445
CV3.65263_LB         0.997761
CV3.65275_LB         0.997935
CV3.65043_LB3.690    0.998034
CV3.65022_LB3.690    0.998119
CV3.65061_LB3.689    0.998268
CV3.65101_LB         0.998317
CV3.65059_LB3.690    0.998531
CV3.65116_LB         0.999686
CV3.65093_LB         1.000000
Name: CV3.65093_LB, dtype: float64

In [101]:
import numpy as np
def diff_submit(tmp):
    for col in tmp.columns:
        tmp[f'd_{col}'] = np.abs(tmp['CV3.65133_LB3.689'] - tmp[col])
    diff_cols = [col for col in tmp.columns if col.count('d_')]    
    display(tmp[diff_cols].sum().sort_values().astype('int'))
    
diff_submit(df)

d_CV3.65133_LB3.689        0
d_CV3.65133_LB             0
d_CV3.65085_LB          3904
d_CV3.65059_LB3.690     4751
d_CV3.65061_LB3.689     4872
d_CV3.65116_LB          4927
d_CV3.65093_LB3.691     5142
d_CV3.65022_LB3.690     5315
d_CV3.65043_LB3.690     5421
d_CV3.65263_LB          5442
d_CV3.65001_LB3.689     5551
d_CV3.65101_LB          5786
d_CV3.65167_LB3.690     6006
d_CV3.65124_LB3.690     6244
d_CV3.64974_LB3.692     6577
d_CV3.65275_LB          7095
d_CV3.65355_LB3.694     8002
d_CV3.65221_LB3.692     8485
d_CV3.64973_LB3.692     8746
d_CV3.64778_LB3.691     8919
d_CV3.65034_LB3.692     9153
d_CV3.64792_LB3.691     9199
d_CV3.65263_LB3.694     9633
d_CV3.64923_LB3.691    11826
d_CV3.64877_LB3.698    13381
d_CV3.64851_LB3.693    17258
dtype: int64